In [12]:
from collections import defaultdict

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def lcm(a, b):
    return a * b // gcd(a, b)

def parse_input(inp, p2mode = False):
    flips = defaultdict(list)
    conjs = defaultdict(list)
    broads = []
    for idx, line in enumerate(inp):
        tokens = line.strip().split()
        if tokens[0].startswith("%"):
            label = tokens[0][1:]
            for t in tokens[2:]:
                flips[label].append(t.strip(","))
        elif tokens[0].startswith("&"):
            label = tokens[0][1:]
            for t in tokens[2:]:
                conjs[label].append(t.strip(","))
        elif tokens[0] == "broadcaster":
            for t in tokens[2:]:
                broads.append(t.strip(","))
    return flips, conjs, broads

def part1(inp):
    flips, conjs, broad = inp
    flip_states = {}
    conj_mem = defaultdict(dict)
    for f in flips.keys():
        flip_states[f] = False
        for o in flips[f]:
            if o in conjs.keys():
                conj_mem[o][f] = False
    for k, v in conjs.items():
        for o in v:
            if o in conjs.keys():
                conj_mem[o][k] = False
    lows = 0
    highs = 0
    for _ in range(1000):
        pulse_queue = [("broadcaster", False, "button")]
        while len(pulse_queue) > 0:
            nxt = pulse_queue.pop(0)
            dst, typ, src = nxt
            if typ == True:
                highs += 1
            else:
                lows += 1
            if dst == "broadcaster":
                for b in broad:
                    pulse_queue.append((b, typ, dst))
            elif dst in flips.keys():
                if typ == False:
                    flip_states[dst] = not flip_states[dst]
                    for f2 in flips[dst]:
                        pulse_queue.append((f2, flip_states[dst], dst))
            elif dst in conjs.keys():
                conj_mem[dst][src] = typ
                snd = not all(v for v in conj_mem[dst].values())
                for f2 in conjs[dst]:
                    pulse_queue.append((f2, snd, dst))
    return lows * highs

def part2(inp):
    flips, conjs, broad = inp
    flip_states = {}
    conj_mem = defaultdict(dict)
    for f in flips.keys():
        flip_states[f] = False
        for o in flips[f]:
            if o in conjs.keys():
                conj_mem[o][f] = False
    for k, v in conjs.items():
        for o in v:
            if o in conjs.keys():
                conj_mem[o][k] = False
    seens = defaultdict(list)
    lows = 0
    highs = 0
    for i in range(100000000000000000000):
        if i % 100000 == 0 and i > 0:
            cycles = []
            for k, v in seens.items():
                v2 = []
                for i in range(len(v) - 1):
                    v2.append(v[i + 1] - v[i])
                cycles.append(v2[0])
            lcm_value = 1
            for x in cycles:
                lcm_value = lcm(lcm_value, x)
            return lcm_value
        if i > 0 and all(v == False for v in flip_states.values()) and all(all(v == False for v in d.values()) for d in conj_mem.values()):
            pass
        pulse_queue = [("broadcaster", False, "button")]
        while len(pulse_queue) > 0:
            nxt = pulse_queue.pop(0)
            dst, typ, src = nxt
            if src in ["bm", "cl", "tn", "dr"] and typ == True:
                seens[src].append(i + 1)
            if typ == True:
                highs += 1
            else:
                lows += 1
            if dst == "rx" and typ == False:
                return i + 1
            if dst == "broadcaster":
                for b in broad:
                    pulse_queue.append((b, typ, dst))
            elif dst in flips.keys():
                if typ == False:
                    flip_states[dst] = not flip_states[dst]
                    for f2 in flips[dst]:
                        pulse_queue.append((f2, flip_states[dst], dst))
            elif dst in conjs.keys():
                conj_mem[dst][src] = typ
                snd = not all(v for v in conj_mem[dst].values())
                for f2 in conjs[dst]:
                    pulse_queue.append((f2, snd, dst))
    return 0

# Read the input file and use the parse_input function
input_file_path = 'input.txt'  # Adjust the file path as needed
with open(input_file_path, 'r') as file:
    input_data = file.readlines()

parsed_input = parse_input(input_data)
print("Part 1 Answer:", part1(parsed_input))
print("Part 2 Answer:", part2(parsed_input))


Part 1 Answer: 684125385
Part 2 Answer: 1


In [15]:
import sys
import re
from copy import deepcopy
from math import gcd
from collections import defaultdict, Counter, deque
import heapq
import math
D = open('input.txt').read().strip()
L = D.split('\n')
G = [[c for c in row] for row in L]
R = len(G)
C = len(G[0])

def lcm(xs):
  ans = 1
  for x in xs:
    ans = (ans*x)//math.gcd(x,ans)
  return ans

TYP = {}

R = {}
for line in L:
  src, dest = line.split('->')
  src = src.strip()
  dest = dest.strip()
  dest = dest.split(', ')
  R[src] = dest
  TYP[src[1:]] = src[0]

def adjust(y):
  if y in TYP:
    return TYP[y]+y
  else:
    return y

FROM = {}
INV = defaultdict(list)
for x,ys in R.items():
  R[x] = [adjust(y) for y in ys]
  for y in R[x]:
    if y[0]=='&':
      if y not in FROM:
        FROM[y] = {}
      FROM[y][x] = 'lo'
    INV[y].append(x)

assert len(INV['rx'])==1
assert INV['rx'][0][0]=='&'
WATCH = INV[INV['rx'][0]]

lo = 0 
hi = 0
Q = deque()
ON = set()
PREV = {}
COUNT = defaultdict(int)
TO_LCM = []
for t in range(1, 10**8):
  Q.append(('broadcaster', 'button', 'lo'))

  while Q:
    x, from_, typ = Q.popleft()

    if typ=='lo':
      # Assume &df -> rx AND &a -> df, &b -> df, etc.
      # We want to know when &a,&b get 'lo' input, because then they will send a 'hi' input to &df
      # When will &df get lo inputs from all of them?
      # Assume that they get lo inputs on a perfect cycle and that df must get a 'hi' input from all of them on the same
      #  timestamp. I assume they get a lot of 'hi' inputs on other timestamps that "reset" them?
      if x in PREV and COUNT[x]==2 and x in WATCH:
        #print(f't={t} cycle={t-PREV[x]} x={x} new_typ={new_typ}')
        TO_LCM.append(t-PREV[x])
      PREV[x] = t
      COUNT[x] += 1
    if len(TO_LCM) == len(WATCH):
      print(lcm(TO_LCM))
      sys.exit(0)

    if x=='rx' and typ=='lo':
      # won't happen; takes too long
      print(t+1)

    if typ=='lo':
      lo += 1
    else:
      hi += 1

    if x not in R:
      continue
    if x=='broadcaster':
      for y in R[x]:
        Q.append((y, x, typ))
    elif x[0]=='%':
      if typ=='hi':
        continue
      else:
        if x not in ON:
          ON.add(x)
          new_typ = 'hi'
        else:
          ON.discard(x)
          new_typ = 'lo'
        for y in R[x]:
          Q.append((y, x, new_typ))
    elif x[0]=='&':
      FROM[x][from_] = typ
      new_typ = ('lo' if all(y=='hi' for y in FROM[x].values()) else 'hi')
      for y in R[x]:
        Q.append((y, x, new_typ))
    else:
      assert False, x
  if t==1000:
    print(lo*hi)


684125385
225872806380073


SystemExit: 0

c:\Users\alira\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
